In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import operator
import dill

## Versuchsbeschreibung
Wählen sie eine Klasse aus. Versuchen Sie zu jeder Klasse eine Entscheidungsfunktion zu finden,
die es dem Rechner ermöglicht, alle Muster richtig zu klassifizieren.

In [5]:
# define font
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16}

selectclass = widgets.Dropdown(
    options=['Summe von W1 und W2 größer als 8', 'W1 ist größer als W2', 'Das Produkt von W1 und W2 ist durch 3 teilbar', 'W1 und W2 sind größer als 3'],
    value='Summe von W1 und W2 größer als 8',
    description='Klasse auswählen:',
    layout={'width': '450px'},
    style={'description_width': '130px'},
    disabled=False)

# init state dict for saving status
save_string = 'save/state_dict_2.dill'
state_dict = {}

# try to load state_dict
try: 
    with open(save_string, 'rb') as fp:
        state_dict = dill.load(fp)
    # set data from load
    x1.value = str(state_dict[selectclass.value]['x1'])
    x2.value = str(state_dict[selectclass.value]['x2'])
    x3.value = str(state_dict[selectclass.value]['x3'])
    select.value = str(state_dict[selectclass.value]['op'])
except:
    # init otherwise
    for option in selectclass.options:
        state_dict[option] = {'x1': 0.0, 'x2': 1.0, 'x3': 0.0, 'op': '>'}

x1 = widgets.Text(value = str(state_dict[selectclass.value]['x1']),
                  placeholder = '',
                  description = '',
                  disabled = False,
                  layout = {'width' : '50px'},
                  continuous_update=True)

x2 = widgets.Text(value = str(state_dict[selectclass.value]['x2']),
                  placeholder = '',
                  description = '',
                  disabled = False,
                  layout = {'width' : '50px'},
                  continuous_update=True)

x3 = widgets.Text(value = str(state_dict[selectclass.value]['x3']),
                  placeholder = '',
                  description = '',
                  disabled = False,
                  layout = {'width' : '50px'},
                  continuous_update=True)

operator_dict = {'>': operator.gt,
                 '<': operator.lt,
                 '>=': operator.ge,
                 '<=': operator.le}

select = widgets.Dropdown(
    options = list(operator_dict.keys()),
    value = state_dict[selectclass.value]['op'],
    disabled = False,
    layout = {'width' : '50px', 'height' : '28px'})


l12 = widgets.Label(value='$W_1^2 +$')
l1 = widgets.Label(value='$W_1 +$')
l2 = widgets.Label(value='$W_2$')
items = [x1, l12, x2, l1, x3, select, l2]
box = widgets.Box(children=items)
display(box)
display(selectclass)

# create figure
fig = plt.figure(figsize=(8, 6))
ax1 = fig.add_subplot(111)
ax1.set_aspect('equal')
ax1.axis([0, 13, 0, 7])
plt.xticks(np.arange(0, 8, 1.0))
plt.tight_layout()
plt.ion()

# define base grid for classes
w1, w2 = np.mgrid[1:7, 1:7]

# plot class boarder func
def plot_separation():
    y = np.linspace(0, 7, num=100)
    y = float(x1.value) * x**2 + float(x2.value) * x + float(x3.value)
    line1.set_data([x, y])

# define class separation
x = np.linspace(0, 7, num=100)
line1, = ax1.plot(x, x, linewidth=1, color='k')
plot_separation()

# init multiple dicts
r_x, r_y, b_x, b_y = ({} for i in range(4))

# exploit the list order of the dropdown widget and create classes
for i, item in enumerate(selectclass.options):
    if i == 0:
        r_x[item], r_y[item] = np.where(w1 + w2 <= 8)
        b_x[item], b_y[item] = np.where(w1 + w2 > 8)
    elif i == 1:
        r_x[item], r_y[item] = np.where(w1 <= w2)
        b_x[item], b_y[item] = np.where(w1 > w2)
    elif i == 2:
        r_x[item], r_y[item] = np.where((w1 * w2) % 3 != 0)
        b_x[item], b_y[item] = np.where((w1 * w2) % 3 == 0)
    elif i == 3:
        r_x[item], r_y[item] = np.where(~((w1 > 3) & (w2 > 3)))
        b_x[item], b_y[item] = np.where((w1 > 3) & (w2 > 3))
        
# init graph 
red_dots, = ax1.plot(r_x[selectclass.value] + 1, r_y[selectclass.value] + 1, 
                    'ro', color = 'r', markersize = 15) # +1 offset for plot
blue_dots, = ax1.plot(b_x[selectclass.value] + 1, b_y[selectclass.value] + 1, 
                     'ro', color = 'b', markersize = 15) # +1 offset for plot

# init markers
marker1, = ax1.plot(-1, -1, 'ro', color = 'r', markersize = 15, markeredgecolor='blue')
marker2, = ax1.plot(-1, -1, 'ro', color = 'b', markersize = 15, markeredgecolor='red')

# init text fields
right_text = ax1.text(8, 6, 'Richtig erkannt: %d' % 0, fontdict=font)
f_p_text = ax1.text(8, 5.5, 'False positives: %d' % 0, fontdict=font)
f_n_text = ax1.text(8, 5, 'False negatives: %d' % 0, fontdict=font)

# plot labels axis
plt.xlabel('W1')
plt.ylabel('W2')
plt.gcf().subplots_adjust(left=0.05)
ax1.xaxis.set_label_coords(0.25, -0.06)

# save data
def save_data():
    state_dict[selectclass.value]['x1'] = float(x1.value)
    state_dict[selectclass.value]['x2'] = float(x2.value)
    state_dict[selectclass.value]['x3'] = float(x3.value)
    state_dict[selectclass.value]['op'] = select.value
    with open(save_string, 'wb') as fp:
        dill.dump(state_dict, fp)

# classify data
def classify_data():
    # check reclassification  
    classify = float(state_dict[selectclass.value]['x1']) * w1**2 + float(state_dict[selectclass.value]['x2']) * w1 +\
                float(state_dict[selectclass.value]['x3']) - w2

    # calculate error of reclassification
    # red markers
    r_x_c, r_y_c = np.where(~operator_dict[select.value](classify, 0.0))
    list_red = list(zip(r_x_c, r_y_c))
    red_filter_out = list(zip(b_x[selectclass.value], b_y[selectclass.value]))
    red_plot = [x for x in list_red if x not in red_filter_out]
    
    # print if possible
    if red_plot:
        r_x_plot, r_y_plot = map(np.array, zip(*red_plot))
        marker1.set_visible(True)
        marker1.set_data(r_x_plot + 1, r_y_plot + 1)
    else:
        marker1.set_visible(False)
    
    # blue markers
    b_x_c, b_y_c = np.where(operator_dict[select.value](classify, 0.0))
    list_blue = list(zip(b_x_c, b_y_c))
    blue_filter_out = list(zip(r_x[selectclass.value], r_y[selectclass.value]))
    blue_plot = [x for x in list_blue if x not in blue_filter_out]
    
    # print if possible
    if blue_plot:
        b_x_plot, b_y_plot = map(np.array, zip(*blue_plot))
        marker2.set_visible(True)
        marker2.set_data(b_x_plot + 1, b_y_plot + 1)
    else:
        marker2.set_visible(False)
    
    # print right recognition, false positives and false negatives
    right_text.set_text('Richtig erkannt: %d' % (w1.size - len(red_plot) - len(blue_plot)))
    f_p_text.set_text('False positives: %d' % len(blue_plot))
    f_n_text.set_text('False negatives: %d' % len(red_plot))
        
    
        
# plot markers
classify_data()

# plot graph based on dropdown selection
def on_change_class(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # set data dict
        x1.value = str(state_dict[change['new']]['x1'])
        x2.value = str(state_dict[change['new']]['x2'])
        x3.value = str(state_dict[change['new']]['x3'])
        select.value = str(state_dict[change['new']]['op'])
        red_dots.set_data(r_x[change['new']] + 1, r_y[change['new']] + 1)
        blue_dots.set_data(b_x[change['new']] + 1, b_y[change['new']] + 1)
        classify_data()
        plot_separation()
            
def on_change_operator(change):
    # save data
    save_data()
    # plot update
    classify_data()
    plot_separation()

    

def callback(wdgt):
    # save data
    save_data()
    try:
        plot_separation()
        classify_data()
    except ValueError:
        print('Please use . instead of ,')
        return
    
# update graph when hitting submit
x1.on_submit(callback)
x2.on_submit(callback)   
x3.on_submit(callback)

# change graph based on "Klasse auswählen"
selectclass.observe(on_change_class)
select.observe(on_change_operator)

Box(children=(Text(value='0.0', layout=Layout(width='50px'), placeholder=''), Label(value='$W_1^2 +$'), Text(v…

Dropdown(description='Klasse auswählen:', layout=Layout(width='450px'), options=('Summe von W1 und W2 größer a…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 1. Welche der Klassifikatoren sind nicht-linear?

In [7]:
save_checkbox = 'save/checkbox_array.dill'

checkbox_array = np.array([False, False, False, False])
# try to load checkbox array
try: 
    with open(save_checkbox, 'rb') as fp:
        checkbox_array = dill.load(fp)
except:
    pass

checkbox_list = ['Polynom-Klassifikatoren', 'Dynamic Time Warp', 
                 'Support-Vector-Machines', 'Künstliche neuronale Netze']

def callback_checkbox(change):
    if change['type'] == 'change' and change['name'] == 'value':
        checkbox_array[checkbox_list.index(change['owner'].description)] = change['owner'].value
        # save when changed
        with open(save_checkbox, 'wb') as fp:
            dill.dump(checkbox_array, fp)
        
checkbox_dict = {element:widgets.Checkbox(
    value=bool(checkbox_array[i]),
    description=element,
    disabled=False) for i, element in enumerate(checkbox_list)}

for ele in checkbox_list:
    checkbox_dict[ele].observe(callback_checkbox)

display(checkbox_dict['Polynom-Klassifikatoren'], checkbox_dict['Dynamic Time Warp'],
        checkbox_dict['Support-Vector-Machines'], checkbox_dict['Künstliche neuronale Netze'])


Checkbox(value=True, description='Polynom-Klassifikatoren')

Checkbox(value=True, description='Dynamic Time Warp')

Checkbox(value=False, description='Support-Vector-Machines')

Checkbox(value=True, description='Künstliche neuronale Netze')

In [8]:
# Autograding answer, please ignore
